In [1]:
import csv
from decimal import Decimal
import numpy as np
import pandas as pd

# 根拠

準備

一次エネルギー換算係数

9760 kJ/kWh

In [2]:
f_prim = 9760.0

月ごとの日数・始点・終点

In [3]:
d_m = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
d_end = np.cumsum(d_m)
d_str = np.insert(d_end[0: -1], 0, 0)
d_str, d_end

(array([  0,  31,  59,  90, 120, 151, 181, 212, 243, 273, 304, 334],
       dtype=int32),
 array([ 31,  59,  90, 120, 151, 181, 212, 243, 273, 304, 334, 365],
       dtype=int32))

## 機械換気設備の参照一次エネルギー消費量

機械換気設備の参照エネルギー消費量の計算方法を整理するにあたり、建築物省エネ法の計算方法（第5章「換気設備」）（以下、省エネ法第5章という。）をレビューする。日付$d$の時刻$t$における1時間当たりの機械換気設備の消費電力量$E_{E,V,d,t}$は、省エネ法第5章の式(1)に基づき、次式により表される。

$$ E_{E,V,d,t} = E_{E,VG,d,t} + E_{E,VL,d,t} \tag{1} $$

$E_{E,V,d,t}$	：日付$d$の時刻$t$における1時間当たりの機械換気設備の消費電力量, kWh/h  
$E_{E,VG,d,t}$	：日付$d$の時刻$t$における1時間当たりの全般換気設備の消費電力量, kWh/h  
$E_{E,VL,d,t}$	：日付$d$の時刻$t$における1時間当たりの局所換気設備の消費電力量, kWh/h

日付$d$の時刻$t$における1時間当たりの全般換気設備の消費電力量$E_{E,VG,d,t}$は、省エネ法第5章の式(2)に基づき、次式により表される。

$$ E_{E,VG,d,t} = f_{SFP,d,t} \cdot V_R \cdot 10^{-3} \tag{2} $$

$f_{SFP,d,t}$	：日付$d$の時刻$t$における全般換気設備の比消費電力, W/(m<sup>3</sup>/h))  
$V_R$	：全般換気設備の参照機械換気量, m<sup>3</sup>/h

全般換気設備の参照機械換気量$V_R$は、当該住戸の床面積の合計、参照天井高さ、換気回数及び全般換気設備の換気量の余裕率の積を、有効換気量率で除した値であり、省エネ法第5章の式(3)に基づき、次式により表される。

$$ V_R = \frac{ A_A \cdot H_R \cdot N \cdot a }{ e } \tag{3} $$

$A_A$	：床面積の合計, m<sup>2</sup>  
$H_R$	：参照天井高さ, m (= 2.4 m)  
$N$	：換気回数, 1/h  
$a$	：全般換気設備の換気量の余裕率  
$e$	：有効換気量率  

参照天井高さ$H_R$は、2.4mに等しいとされている。  
換気回数$N$は基準値の計算においては0.5回/hである。  
全般換気設備の換気量の余裕率$a$は1.1に等しいとされている。  
日付$d$の時刻$t$における比消費電力$f_{SFP,d,t}$は、日付$d$の時刻$t$に依らず同じ値$f_{SFP}$に等しいとし、基準値の計算においては0.3 W/(m<sup>3</sup>/h)とされている。  
有効換気量率$e$は基準値の設定が第一種換気ではないことから1.0とする。  
これらを勘案すると、

$$
\begin{align*}
E_{E,VG,d,t} &= f_{SFP,d,t} \cdot V_R \cdot 10^{-3} \\
&= \frac{ f_{SFP,d,t} \cdot A_A \cdot H_R \cdot N \cdot a }{ e } \cdot 10^{-3} \\
&= 0.3 \cdot 2.4 \cdot 0.5 \cdot 1.1 \cdot 10^{-3} \cdot A_A \\
&= 0.396 \cdot 10^{-3} \cdot A_A \\
\end{align*}
\tag{4}
$$

In [4]:
# kWh/m2
0.3 * 2.4 * 0.5 * 1.1 / 1.0 * 0.001

0.00039600000000000003

となり、床面積の合計に$0.396⋅10^{-3}$を乗じることで全般換気設備の消費電力量が求まることが分かる。  
月別の消費量については、次式で計算できるとする。

$$ E_{E,VG,m} = a' \cdot A_A \tag{5} $$



この係数$a'$は、係数 $0.396 \cdot 10^{-3}$に月ごとの時間数を乗じて、次表となる。


表1 係数$a'$

In [5]:
coeff_v_g = d_m * 0.396 * 0.001 * 24
pd.DataFrame(coeff_v_g, columns=['月別の係数 alpha_dsh, kWh/m2'], index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月'])


,"月別の係数 alpha_dsh, kWh/m2"
1月,0.294624
2月,0.266112
3月,0.294624
4月,0.285120
5月,0.294624
6月,0.285120
7月,0.294624
8月,0.294624
9月,0.285120
10月,0.294624


局所換気設備の消費電力は省エネ法第5章の式(5)において居住人数で按分することになっているが、今回の用途分解では居住人数を指定することにより特定できることから、按分する式は採用しない。

居住人数・在室スケジュールごとに定められた時々刻々の局所換気の消費電力を読み込み、日積算値に変換する。（その過程で2進数の都合で端数がでるため値をまるめる。）

In [6]:
weekday_raw = []
holiday_out_raw = []
holiday_in_raw = []

with open('ventilation_local.csv') as f:
    reader = csv.reader(f, delimiter='\t')
    rows = [row for row in reader]

    weekday_raw.append(sum([float(row[1]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[2]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[3]) for row in rows[:24]]))
    weekday_raw.append(sum([float(row[4]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[5]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[6]) for row in rows[:24]]))
    weekday_raw.append(sum([float(row[7]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[8]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[9]) for row in rows[:24]]))
    weekday_raw.append(sum([float(row[10]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[11]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[12]) for row in rows[:24]]))

weekday = [Decimal(str(v)).quantize(Decimal('0.01')) for v in weekday_raw]
holiday_out = [Decimal(str(v)).quantize(Decimal('0.01')) for v in holiday_out_raw]
holiday_in = [Decimal(str(v)).quantize(Decimal('0.01')) for v in holiday_in_raw]


居住人数・在室スケジュールごとの日積算値は、次表のようになる。

表2 局所換気設備の消費電力, Wh/d

In [7]:
pd.DataFrame(
    [weekday, holiday_out, holiday_in],
    columns=['居住人数1人', '居住人数2人', '居住人数3人', '居住人数4人'],
    index=['在室スケジュール「平日」', '在室スケジュール「休日外出」', '在室スケジュール「休日在宅」']
)

,居住人数1人,居住人数2人,居住人数3人,居住人数4人
在室スケジュール「平日」,36.94,73.81,110.77,147.61
在室スケジュール「休日外出」,20.59,41.02,61.62,82.02
在室スケジュール「休日在宅」,40.33,80.59,120.95,161.18


これを建築物省エネ法の計算方法（第11章「その他」第1節「地域の区分と外気条件」）（以下、省エネ法第11章第1節という。）で規定される年間スケジュールに従って展開し、月ごとに積算する。

年間のスケジュールを読み込む。

In [8]:
vent_schedule = None
with open('schedule.csv', encoding='utf8') as f:
    reader = csv.reader(f, delimiter='\t')
    rows = [row for row in reader]
    vent_schedule = [
        {
            '平日': 'W',
            '休日外': 'HO',
            '休日在': 'HI'
        }[vs[2]] for vs in rows[1:366]
    ]
vent_schedule = np.array(vent_schedule)
print(vent_schedule)

['HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W'
 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W'
 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W'
 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W'
 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'HI' 'W' 'W' 'W' 'HI'
 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W'
 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W'
 'HI' 'HO' 'W' 'W' 'HO' 'HO' 'HI' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO'
 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W'
 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI'
 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W'
 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'HI' 'W'
 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W'
 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 

日付dの時間tにおける局所換気の消費電力を整理する。

In [9]:
e_e_vl_1_d_t = np.zeros(shape=365, dtype=float)
e_e_vl_2_d_t = np.zeros(shape=365, dtype=float)
e_e_vl_3_d_t = np.zeros(shape=365, dtype=float)
e_e_vl_4_d_t = np.zeros(shape=365, dtype=float)

e_e_vl_1_d_t[vent_schedule=='W'] = weekday[0]
e_e_vl_1_d_t[vent_schedule=='HI'] = holiday_in[0]
e_e_vl_1_d_t[vent_schedule=='HO'] = holiday_out[0]

e_e_vl_2_d_t[vent_schedule=='W'] = weekday[1]
e_e_vl_2_d_t[vent_schedule=='HI'] = holiday_in[1]
e_e_vl_2_d_t[vent_schedule=='HO'] = holiday_out[1]

e_e_vl_3_d_t[vent_schedule=='W'] = weekday[2]
e_e_vl_3_d_t[vent_schedule=='HI'] = holiday_in[2]
e_e_vl_3_d_t[vent_schedule=='HO'] = holiday_out[2]

e_e_vl_4_d_t[vent_schedule=='W'] = weekday[3]
e_e_vl_4_d_t[vent_schedule=='HI'] = holiday_in[3]
e_e_vl_4_d_t[vent_schedule=='HO'] = holiday_out[3]

月ごとに積算する。（小数点を2桁までに丸めてある。）

In [10]:
e_e_vl_1_m = [np.around(e_e_vl_1_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]
e_e_vl_2_m = [np.around(e_e_vl_2_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]
e_e_vl_3_m = [np.around(e_e_vl_3_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]
e_e_vl_4_m = [np.around(e_e_vl_4_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]

居住人数ごとに計算してきた値を縦方向：月、横方向：居住人数の2次元配列にする。

In [11]:
e_e_vl_m = np.array([e_e_vl_1_m, e_e_vl_2_m, e_e_vl_3_m, e_e_vl_4_m]).T

表3 局所換気設備の消費電力（月別）, Wh

In [12]:
pd.DataFrame(
    e_e_vl_m,
    columns=['居住人数1人', '居住人数2人', '居住人数3人', '居住人数4人'],
    index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月']
)


,居住人数1人,居住人数2人,居住人数3人,居住人数4人
1月,1136.17,2269.99,3406.83,4539.72
2月,1021.96,2041.78,3064.34,4083.32
3月,1136.17,2269.99,3406.83,4539.72
4月,1082.88,2163.39,3246.91,4326.52
5月,1103.47,2204.41,3308.53,4408.54
6月,1095.84,2189.40,3285.88,4378.54
7月,1142.95,2283.55,3427.19,4566.86
8月,1132.78,2263.21,3396.65,4526.15
9月,1082.88,2163.39,3246.91,4326.52
10月,1119.82,2237.20,3357.68,4474.13


表1と表3に一次エネ換算係数を乗じて一次エネルギー消費量を求める式になおすと以下のようになる。

$$E_{V,m} = \alpha \cdot A_A + \beta$$

$E_{V,m}$	：月$m$における換気設備の一次エネルギー消費量, MJ  
$\alpha$    ：係数$\alpha$, MJ/m2  
$\beta$    ：係数$\beta$, MJ  

係数α及び係数βは表4及び表5の値とする。

表4 係数 $\alpha$, MJ/m2

In [13]:
alpha = coeff_v_g * f_prim * 0.001

In [14]:
pd.DataFrame(alpha, columns=['月別の係数 alpha, MJ/m2'], index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月'])

,"月別の係数 alpha, MJ/m2"
1月,2.875530
2月,2.597253
3月,2.875530
4月,2.782771
5月,2.875530
6月,2.782771
7月,2.875530
8月,2.875530
9月,2.782771
10月,2.875530


係数 $\beta$

MJ

In [15]:
beta = e_e_vl_m * 0.001 * f_prim * 0.001

In [16]:
pd.DataFrame(
    beta,
    columns=['居住人数1人', '居住人数2人', '居住人数3人', '居住人数4人'],
    index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月']
)

,居住人数1人,居住人数2人,居住人数3人,居住人数4人
1月,11.089019,22.155102,33.250661,44.307667
2月,9.974330,19.927773,29.907958,39.853203
3月,11.089019,22.155102,33.250661,44.307667
4月,10.568909,21.114686,31.689842,42.226835
5月,10.769867,21.515042,32.291253,43.027350
6月,10.695398,21.368544,32.070189,42.734550
7月,11.155192,22.287448,33.449374,44.572554
8月,11.055933,22.088930,33.151304,44.175224
9月,10.568909,21.114686,31.689842,42.226835
10月,10.929443,21.835072,32.770957,43.667509


In [17]:
alpha

array([2.87553024, 2.59725312, 2.87553024, 2.7827712 , 2.87553024,
       2.7827712 , 2.87553024, 2.87553024, 2.7827712 , 2.87553024,
       2.7827712 , 2.87553024])

In [18]:
beta

array([[11.0890192, 22.1551024, 33.2506608, 44.3076672],
       [ 9.9743296, 19.9277728, 29.9079584, 39.8532032],
       [11.0890192, 22.1551024, 33.2506608, 44.3076672],
       [10.5689088, 21.1146864, 31.6898416, 42.2268352],
       [10.7698672, 21.5150416, 32.2912528, 43.0273504],
       [10.6953984, 21.368544 , 32.0701888, 42.7345504],
       [11.155192 , 22.287448 , 33.4493744, 44.5725536],
       [11.0559328, 22.0889296, 33.151304 , 44.175224 ],
       [10.5689088, 21.1146864, 31.6898416, 42.2268352],
       [10.9294432, 21.835072 , 32.7709568, 43.6675088],
       [10.7615712, 21.5008896, 32.2689024, 42.9994368],
       [10.9294432, 21.835072 , 32.7709568, 43.6675088]])